In [ ]:
%%sh
pip install -q pip --upgrade
pip install -q sagemaker --upgrade --user

### Training a Keras CNN on Fashion-MNIST with TensorFlow 2.0

Fashion-MNIST is a Zalando dataset consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. It's a drop-in replacement for MNIST.

https://github.com/zalandoresearch/fashion-mnist/

In this notebook, we'll train a simple CNN built with Keras, using the built-in Tensorflow and Apache MXNet containers provided by Amazon SageMaker.

In [ ]:
import sagemaker
print(sagemaker.__version__)

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

## Download the Fashion-MNIST dataset

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist
(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

In [ ]:
!pygmentize mnist.py

## Train with Tensorflow on the notebook instance (aka 'local mode')

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist.py',
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='2.0.0', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )

In [ ]:
tf_estimator.fit({'training': 'file://data', 'validation': 'file://data'})

## Upload Fashion-MNIST data to S3

In [ ]:
prefix = 'keras-fashion-mnist'

training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

## Train with Tensorflow on a GPU instance

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist.py',
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='2.0.0', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 10}
                         )

In [ ]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

## Deploy

In [ ]:
import time

tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tf_estimator.deploy(
                 initial_instance_count=1, 
                 instance_type='ml.m4.xlarge',
                 endpoint_name=tf_endpoint_name)

## Predict 

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = tf_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

## Clean up

In [ ]:
sess.delete_endpoint(endpoint_name=tf_endpoint_name)